In [1]:
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd

funktion dnb_sru nimmt den paramter 'query' der sru abfrage entgegen und liefert alle ergebnisse als eine liste von records

In [2]:
def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query    
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content)
    records = xml.find_all('record', {'type':'Bibliographic'})
    
    if len(records) < 100:
        
        return records
    
    else:
        
        num_results = 100
        i = 101
        while num_results == 100:
            
            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content)
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records

funktion parse_records nimmt als parameter jeweils ein record-xml entgegen und sucht über xpath gewünschte informationen heraus und liefert diese als dictionary zurück.

In [3]:
def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    
    # umfang
    umfang = xml.xpath("marc:datafield[@tag = '300']/marc:subfield[@code = 'a']", namespaces=ns)
    
    try:
        umfang = umfang[0].text
        #umfang = unicodedata.normalize("NFC", umfang)
    except:
        umfang = "unkown"
        
    meta_dict = {"idn":idn,
                 "umfang":umfang}
    
    return meta_dict

In [10]:
records = dnb_sru('partOfTreasury=1137954264')
print(len(records), 'Ergebnisse')

50 Ergebnisse


In [11]:
#output ist list of dictionaries, die in ein df umgewandelt wird
output = [parse_record(record) for record in records]
df = pd.DataFrame(output)
df

,idn,umfang
0,1154120503,1 Ausstellungsverzeichnis
1,1154120481,1 Ausstellungsverzeichnis
2,115412049X,1 Foto-Negativ
3,1228187452,1 Blatt
4,1154125408,1 Ex.
5,1154104192,2 S. (Kopie)
6,1154104095,2 Ex. (Kopien)
7,1154113566,16 Bg. (der komplette Text) mit Schutzumschlag
8,1154113760,16 Bögen
9,1154113744,45 Blätter
